
# <span style="color:DarkSeaGreen">JumpStart Lab 2</span>

This lab does the following:

- Uses the endpoint created in Lab 1
- Implements SageMaker application-autoscaling
- Tests the functionality 



# <span style="color:DarkSeaGreen">Prepare Your Environment</span>
### Note if you want a venv, see Lab 1

# Lab 2 Starts Here!

# <span style="color:DarkSeaGreen">Setup</span>

In [ ]:
# region
# for the purpose of this lab, us-east-1, us-west-2, eu-west-1 has the broadest coverage of JumpStart models and instance types
# if you provision in other regions, you may not have access to all the models or instance types, and may need to request increase of quotas for some instance types
myRegion='us-east-1'

# parameter store
myParameterStoreEndpointName='doit-jumpstart-sagemaker-endpoint-name'
myParameterStoreIAMARN='doit-jumpstart-sagemaker-iam-arn'

print ('Done! Move to the next cell ->')

In [ ]:
# import libraries
import boto3
from certifi import where

botoSession = boto3.Session(region_name=myRegion)

# Configure boto3 to use certifi's certificates - helps avoid SSL errors if your system’s certificate store is out of date or missing root certs
sts_client = boto3.client('sts', verify=where())
myAccountNumber = sts_client.get_caller_identity()["Account"]
print(myAccountNumber)
print(sts_client.get_caller_identity()["Arn"])

# create clients we can use later
# iam
iam = boto3.client('iam', region_name=myRegion, verify=where())
# ssm
ssm = boto3.client('ssm', region_name=myRegion, verify=where())

print ('Done! Move to the next cell ->')

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": "doit-jumpstart"},
    {"Key": "project", "Value": "lab1"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": "doit-jumpstart",
    "project": "lab1",
    "author": "simon",
}

print ('Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">IAM</span>

In [ ]:
def getSageMakerExecutionRole():
    """
    Gets a role required for SageMaker to run jobs on your behalf
    Only needed if this is being run in a local IDE, not needed if in SageMaker Studio or SageMaker Notebook Instance

    Args:
        None

    Returns:
        An IAM execution role ARN
    """

    # get the role we created in the previous lab from the parameter store
    response = ssm.get_parameter(Name=myParameterStoreIAMARN)
    myRoleSageMakerExecutionARN = response['Parameter']['Value']
    print(f"Retrieved role from parameter store: {myRoleSageMakerExecutionARN}")    

    return myRoleSageMakerExecutionARN

# <span style="color:DarkSeaGreen">Get Execution Role and Session</span>
- SageMaker requires an execution role to assume on your behalf

In [ ]:
from sagemaker.session import get_execution_role
from sagemaker.session import Session

try:
    # if this is being run in a SageMaker AI JupyterLab Notebook
    myRoleSageMakerExecutionARN = get_execution_role()
except:
    # if this is being run in a local IDE - we need to create our own role
    myRoleSageMakerExecutionARN = getSageMakerExecutionRole()

# make sure we get a session in the correct region (needed as it can use the aws configure region if running this locally
sageMakerSession = Session(boto_session=botoSession)

print(myRoleSageMakerExecutionARN)
print(sageMakerSession)

print ('Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">Get the Endpoint from Lab 1</span>

In [ ]:
# get the endpoint created in lab1
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# get the endpoint name from parameter store
response = ssm.get_parameter(
    Name=myParameterStoreEndpointName
)
endpoint_name = response['Parameter']['Value']
print(f"Using endpoint name: {endpoint_name}")  

# create a predictor to interact with the endpoint - need to specify the default serializer and deserializer this time
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sageMakerSession,
    serializer=JSONSerializer(),      
    deserializer=JSONDeserializer()
)

print ('Done! Move to the next cell ->')

In [ ]:
# test the endpoint
example_payloads = [
    {
        "body": {
            "inputs": "Write a Python function to check if a number is prime",
            "parameters": {"max_new_tokens": 128, "temperature": 0.2, "top_p": 0.9},
        },
        "content_type": "application/json",
        "accept": "application/json",
    },
    {
        "body": {
            "inputs": "Describe what a llm model can do for someone who is sceptical about them",
            "parameters": {"max_new_tokens": 128, "temperature": 0.2, "top_p": 0.9},
        },
        "content_type": "application/json",
        "accept": "application/json",
    },
]


for payload in example_payloads:
    body = payload.body if hasattr(payload, "body") else payload["body"]
    response = predictor.predict(body)
    response = response[0] if isinstance(response, list) else response
    print("Input:\n", body, end="\n\n")
    print("Output:\n", response["generated_text"].strip(), end="\n\n\n")

print ('Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">Create Scalability Plan</span>

In [ ]:
# https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/huggingfacetgi/meta-llama/llama3-8b/faster-autoscaling/realtime-endpoints/FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb
# https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-inference-launches-faster-auto-scaling-for-generative-ai-models/
# https://docs.aws.amazon.com/autoscaling/application/userguide/what-is-application-auto-scaling.html





# <span style="color:DarkSeaGreen">Clean Up Architecture</span>
### <span style="color:Red">Only do this if you have finished with this lab and any labs that depend on it!</span>
##### It will delete all architecture created, make sure you no longer need any of it!!!

In [ ]:
# when finished with the endpoint, delete it
# This endpoint is deleted in lab 1
# Remember to delete the architecture in lab 1 too